In [ ]:
# !pip install imageio

In [ ]:
# !python train.py -h

In [ ]:
# files modified
"""
Pluralistic/train.py
Pluralistic/options/base_options.py
Pluralistic/dataloader/data_loader.py
"""
# --continue_train

In [1]:
from object_detect.generate_masks import pick_make_bg_img

In [9]:
img_dir = 'linmao-new_txt/JPEGImages'
label_dir = 'linmao-new_txt/Annotations'

In [10]:
pick_make_bg_img(img_dir, label_dir, 'BackgroundCandidates', 8, max_width=0.3, max_height=0.3, max_obj_num=20)

100%|██████████| 8407/8407 [00:01<00:00, 8138.04it/s]


In [5]:
from object_detect.generate_masks import create_rand_mask, create_mask_for_object

In [ ]:
img_dir = 'linmao-test_txt/JPEGImages'
label_dir = 'linmao-test_txt/Annotations'
save_dir = 'linmao-test_txt/Masks'
save_txt_dir = 'linmao-test_txt/Masks_txt'
create_rand_mask(img_dir, label_dir, save_dir, save_txt_dir, num_per_img=6,
                     width_minmax=(0.05, 0.4),
                     hight_minmax=(0.05, 0.4),
                     img_type='jpg',
                     save_type='jpg')

In [11]:
save_obj_mask_dir = 'MaskData/ObjMasks'
create_mask_for_object('BackgroundCandidates/images', 
                       'BackgroundCandidates/labels', save_obj_mask_dir, expand_ratio_width=0, expand_ratio_hight=0, img_type='jpg',
                save_type='jpg')

100%|██████████| 286/286 [00:08<00:00, 31.82it/s]


In [ ]:
from object_detect.box_image_text import plot_box

In [ ]:
import os
out_put_dir = 'linmao-test_txt/Masks_on_origin'
os.makedirs(out_put_dir, exist_ok=True)
plot_box(img_dir, save_txt_dir, out_put_dir, thickness=20, filter_diff=True)

# Note
### Put mask_type as 3 and mask_file in order to use my own masks, the image name and mask name should match

In [ ]:
!python train.py --name linmao --img_file linmao-test_txt/JPEGImages --mask_file linmao-test_txt/Masks --niter 100000 --save_iters_freq 100000 --save_latest_freq 1 --batchSize 4 --mask_type 3 --continue_train



In [ ]:
# !python test.py -h

# batchSize must be 1 in order to get correct predictions
# Modify Pluralistic/options/test_options.py how_many to set up how many pictures to predict

In [ ]:
!python test.py  --name linmao --img_file linmao-test_txt/JPEGImages --mask_file linmao-test_txt/Masks --mask_type 3 --batchSize 1


In [14]:
!python test.py  --name linmao --img_file BackgroundCandidates/images --mask_file MaskData/ObjMasks --mask_type 3 --batchSize 1 --how_many 286


--------------Options--------------
batchSize: 1
checkpoints_dir: ./checkpoints
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 256
fineSize: [512, 512]
gpu_ids: 0
how_many: 286
img_file: BackgroundCandidates/images
isTrain: False
loadSize: [522, 522]
mask_file: MaskData/ObjMasks
mask_type: 3
model: pluralistic
nThreads: 8
name: linmao
no_augment: False
no_flip: False
no_rotation: False
no_shuffle: False
nsampling: 1
ntest: inf
output_scale: 4
phase: test
resize_or_crop: resize_and_crop
results_dir: ./results/
save_number: 1
which_iter: latest
----------------End----------------
testing images = 286
ResEncoder(
  (block0): ResBlockEncoderOptimized(
    (conv1): SpectralNorm(
      (module): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv2): SpectralNorm(
      (module): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (bypass): SpectralNorm(
      (module): Conv2d(3, 32, kernel_size=(1, 1), strid

In [18]:
from PIL import Image
from tqdm import tqdm
import os

def get_inpainted_results(result_dir, save_dir):
    image_path = []
    os.makedirs(save_dir, exist_ok=True)

    for root, dirs, files in os.walk(result_dir):
        for file in files:
            if file.endswith(f".png") and '_out' in file:
                if 'checkpoint' in file:
                    pass
                else:
                    file_path = os.path.join(root, file)
                    image_path.append(file_path)
    for img_p in tqdm(image_path):
        name = os.path.splitext(img_p.split(os.sep)[-1])[0].replace("_out_0", "")
        im1 = Image.open(img_p)
        im1.save(os.path.join(save_dir, name+'.jpg'))
        
    


In [19]:
get_inpainted_results('results', 'background_images')

100%|██████████| 286/286 [00:04<00:00, 64.05it/s]


In [ ]:
import torchvision.transforms as transforms
from PIL import Image, ImageFile

In [ ]:
loadSize = [522, 522]
fineSize = [512, 512]

In [ ]:
transform_list = []
osize = [loadSize[0], loadSize[1]]
fsize = [fineSize[0], fineSize[1]]

transform_list.append(transforms.Resize(fsize))

transform_list += [transforms.ToTensor()]
transform = transforms.Compose(transform_list)

In [ ]:
img_path = 'linmao-test_txt/JPEGImages/nl_0438_0420.jpg'
img_pil = Image.open(img_path).convert('RGB')
img = transform(img_pil)

In [ ]:
import numpy as np
np.array(img_pil).shape

In [ ]:
img.shape

In [ ]:
import cv2


image = img.numpy()
image = cv2.cvtColor(image)

pixels = np.array(image)

plt.imshow(pixels)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
img = img.numpy()
# img = img.reshape(img.shape[1], img.shape[2], img.shape[0])
from matplotlib import pyplot as plt
plt.imshow(img, interpolation='nearest')

In [ ]:
import torch
import os
import numpy as np
import math
from tqdm import tqdm
import random
# image read libraries
from PIL import Image
import matplotlib.pyplot as plt
import shutil

In [ ]:
def pick_make_bg_img(img_dir, label_dir, out_dir, target_label, max_width=0.1, max_height=0.1, max_obj_num=10):
    image_path = []
    label_path = []
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
        os.makedirs(out_dir)
    else:
        os.makedirs(out_dir)

    os.makedirs(os.path.join(out_dir, 'images'))
    os.makedirs(os.path.join(out_dir, 'labels'))

    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if file.endswith(".jpg"):
                if 'checkpoint' in file:
                    pass
                else:
                    file_path = os.path.join(root, file)
                    image_path.append(file_path)

    for root, dirs, files in os.walk(label_dir):
        for file in files:
            if file.endswith(".txt"):
                if 'checkpoint' in file:
                    pass
                else:
                    file_path = os.path.join(root, file)
                    label_path.append(file_path)

    image_path = sorted(image_path)
    label_path = sorted(label_path)
    image_label_path_pair = list(zip(image_path, label_path))

    epsilon = 1e-6
    for im_p, lb_p in tqdm(image_label_path_pair):
        im_name = os.path.splitext(im_p.split(os.sep)[-1])[0]
        with open(lb_p) as f:
            lines = f.read().splitlines()
        all_targets = set()
        w_max = 0
        h_max = 0
        obj_count = 0
        for l in lines:
            line_split = l.split(' ')
            label = int(line_split[0])
            all_targets.add(label)
            x, y, w, h = tuple([float(i) + epsilon for i in line_split[1:5]])
            if w > w_max:
                w_max = w
            if h > h_max:
                h_max = h
            obj_count += 1

        if target_label in all_targets and w_max <= max_width and h_max <= max_height and obj_count<=max_obj_num:
            img_out_path = os.path.join(out_dir, 'images', im_name + '_bg.jpg')
            label_out_path = os.path.join(out_dir, 'labels', im_name + '_bg.txt')
            shutil.copy2(im_p, img_out_path)
            with open(label_out_path, mode='a'): pass

In [ ]:
pick_make_bg_img('linmao-test_txt/JPEGImages', 'linmao-test_txt/Annotations', 
                 'BackGrounds', 8, max_width=0.3, max_height=0.3, max_obj_num=20)